Transformers are just a composable way to process the data. `process` does this one way, `unprocess` does the reverse. Usually `process` means something applied when reading, and `unprocess` means something applied when writing. The terminology is inherited from Kaitai Struct, where `process` key in a dict was used to transform byte arrays before parsing them. The following example teaches you to serialize data using `transformerz` lib.

In [1]:
testJsonDict = {"abolish": ["patent", "copyright"]}

In [2]:
from transformerz.serialization.json import jsonFancySerializer

print(jsonFancySerializer.unprocess(testJsonDict))

{
	"abolish": [
		"patent",
		"copyright"
	]
}


In [3]:
print(jsonFancySerializer.process("[null]"))

[None]


 They can be composed using `+` operation. 

In [4]:
from transformerz.serialization.pon import ponSerializer  # JSON <-> JavaScript === PON <-> Python

print((ponSerializer + jsonFancySerializer).unprocess(testJsonDict))  # Returns a "PON" `str` in which a JSON string is serialized

'{\n\t"abolish": [\n\t\t"patent",\n\t\t"copyright"\n\t]\n}'


In [5]:
print((jsonFancySerializer + ponSerializer).unprocess(testJsonDict))  # Returns a JSON `str` in which "PON" string is serialized

"{'abolish': ['patent', 'copyright']}"


But you cannot save strings into files, you need to save bytes into files ...

In [6]:
from transformerz.text import utf8Transformer

print((utf8Transformer + jsonFancySerializer).unprocess(testJsonDict))  # Returns raw bytes of a JSON string is serialized

b'{\n\t"abolish": [\n\t\t"patent",\n\t\t"copyright"\n\t]\n}'


The data can be compressed. For compression we use the stuff available in my fork of `kaitai.compress` library (I hope it would be merged somewhen). `BinaryProcessor` is an adapter allowing to use the stuff from that lib.

In [7]:
from transformerz.compression import BinaryProcessor
from transformerz.kaitai.compress import Zlib
from transformerz.serialization.cbor import cborSerializer

lzmaProcessor = BinaryProcessor("lzma", Zlib())  # You must name your processor!
print((lzmaProcessor + cborSerializer).unprocess(["fuck"] * 8000))  # Returns ZLIB-compressed BSON-serialized tokens of Spider Jerusalem's reaction on Joe Biden's election

b'x\x9c\xed\xc41\r\x000\x08\x000)\xb3\x82\x97-{\xb8\x11\x83[\x12t\xb4G\xfb\xc4\xfbuS\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\x92$I\xd26q\xa6\x1a\xf9'
